In [25]:
import pickle
import datetime
from blockchain import blockexplorer

In [26]:
dbFile = "simple_loop_process.dat"
timeFile = "times.txt"
timeDiffFile = "times_diff.txt"
actualTimeFile = "actual_times.txt"
blockchains = []
times = []
timeDiffs = []
actual_diffs = []

Reads data from the dbFile. This reads in blocks so we can manipulate them. The type of each block is simpleBlock from blockchain explorer.

In [27]:
def readData():
    with open(dbFile,"rb") as f:
        for _ in range(407908):
            try:
                blockchains.append(pickle.load(f))
            except:
                break

Writes the times to a file. Should only need to be done once, but whatever.

In [28]:
def writeTimes():
    with open(timeFile,"w") as f:
        for value in blockchains:
            f.write(str(value.time)+"\n")
            

Reads in the times from the timeFile.

In [29]:
def readTimes():
    global times
    times = [line.rstrip('\n') for line in open(timeFile)]

In [30]:
readData()

In [40]:
#check that we read in the blocks
len(blockchains)

401470

In [32]:
readTimes()

In [41]:
#check that we read in the times
len(times)

401470

This is a test block. Allows you to examine the timing and info about block. Ignore this as its lame

In [42]:
#print blockchains[0].time
#print blockchains[0].hash
#print blockchains[0].height
#print "########"
#print blockchains[1].time
#print blockchains[1].hash
#print blockchains[1].height
#print "########"
#print int(blockchains[0].time) - int(blockchains[1].time)
a = datetime.datetime.fromtimestamp(1460972535)
b = datetime.datetime.fromtimestamp(1460972545)
print type(a)
print b
#Mon, 18 Apr 2016 09:42:15 GMT
#Mon, 18 Apr 2016 09:42:25 GMT
#print type(blockchains[0].main_chain )
c =  a - b
print c.total_seconds()
print divmod(c.days * 86400 + c.seconds, 60)

<type 'datetime.datetime'>
2016-04-18 05:42:25
-10.0
(-1, 50)


Function that iterates over the times list and computes the difference between two times next to each other. Writes out to time_diff.txt. Reports the block height, time, and whether in main chain for each block as well as the time between each block. The last digit is seconds between blocks. Sample:
```
407864(1460995794,True) - 407863(1460992454,True)==> 0:55:40 ==>3340.0
```

In [37]:
def processTimes():
    global times
    global timeDiffs
    f = open(timeDiffFile,"w")
    for i in range(len(times)):
        if i == len(times)-1:
            break
        #value = int(times[i]) - int(times[i+1])
        value = datetime.datetime.fromtimestamp(int(blockchains[i].time)) - datetime.datetime.fromtimestamp(int(blockchains[i+1].time))
        
        timeDiffs.append(divmod(c.days * 86400 + c.seconds, 60))
        f.write(str(blockchains[i].height)+"("+str(blockchains[i].time)+","+str(blockchains[i].main_chain)+")"+" - "+str(blockchains[i+1].height)+"("+str(blockchains[i+1].time)+","+str(blockchains[i+1].main_chain)+")"+"==> "+str(value)+" ==>"+str(value.total_seconds())+"\n")
    f.close()

In [38]:
processTimes()

In [43]:
#test that we have the right data
len(timeDiffs)

401470

Function that converts unix time to human readable times and writes them to a file. Ignore this

In [44]:
def convertTimes():
    global times
    global actual_diffs
    f = open(actualTimeFile,"w")
    for value in times:
        value = datetime.datetime.fromtimestamp(int(value)).strftime('%Y-%m-%d %H:%M:%S')
        f.write(value+"\n")
    for value in timeDiffs:
        value2 = datetime.datetime.fromtimestamp(int(value)).strftime('%Y-%m-%d %H:%M:%S')
        actual_diffs.append(value2)
    f.close()